In [1]:
import requests
from bs4 import BeautifulSoup
import MySQLdb

db = MySQLdb.connect(host="localhost",  # host      # 127.0.0.1
                     user="root",       # username  # root
                     passwd="1111",     # password  # 1234
                     db="naver_movie")         # DB NAME

db.set_character_set('utf8')

In [2]:
# 데이터베이스 커서(Cursor)는 일련의 데이터에 순차적으로 액세스할 때 검색 및 "현재 위치"를 포함하는 데이터 요소이다.
cur = db.cursor()
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=161967

In [3]:

# IF(조건, 참값, 거짓값)
query = """
        CREATE TABLE IF NOT EXISTS naver_movie
            (code    INTEGER      NOT NULL      PRIMARY KEY,
             created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
             title   VARCHAR(100) NOT NULL,
             story   TEXT,
             released_date  DATETIME)
             ENGINE INNODB;
        """
cur.execute(query)

C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: Warning: (1050, "Table 'naver_movie' already exists")
  if sys.path[0] == '':


0

In [4]:
query = """
        CREATE TABLE IF NOT EXISTS movie_comment(
            score    INTEGER    NOT NULL,
            content TEXT       NOT NULL,
            movie_code INTEGER,
            writer  VARCHAR(100),
            written_at DATETIME,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (writer, written_at),
            FOREIGN KEY(movie_code)
            REFERENCES naver_movie(code) on delete cascade)
            ENGINE INNODB;
            
"""
#on delete cascase 참조된 테이블의 값이 사라지면 같이 사라진다
#on delete None 참조된 테이블의 값이 사라지면 값이 NULL로 변경
cur.execute(query)

C:\Users\15Z970-GA5BK\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: Warning: (1050, "Table 'movie_comment' already exists")


0

In [5]:
import datetime
from urllib import parse
import requests
from bs4 import BeautifulSoup
today = datetime.datetime.now()

def get_movie_list(start_page = 1, end_page = 1):
    URL = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
    params = {
        'sel' :'pnt',
        'date' : 20190616
    }
    movie_list = []
    for i in range(start_page, end_page+1):
        params['page'] = i
        
        resp = requests.get(URL, params)
        soup = BeautifulSoup(resp.content)
        list_ranking = soup.find(class_ = 'list_ranking').find_all('tr')
        
        for movie_tr in list_ranking:
            try:
                code = movie_tr.find('a')['href'].split('=')[-1]
                title = movie_tr.find('a').text
                movie_list.append({
                    'code' : code,
                    'title': title
                })
            except Exception:
                pass
            
    return movie_list


In [6]:
get_movie_list()

[{'code': '171539', 'title': '그린 북'},
 {'code': '174830', 'title': '가버나움'},
 {'code': '144906', 'title': '베일리 어게인'},
 {'code': '163788', 'title': '알라딘'},
 {'code': '169240', 'title': '아일라'},
 {'code': '151196', 'title': '원더'},
 {'code': '157243', 'title': '당갈'},
 {'code': '156464', 'title': '보헤미안 랩소디'},
 {'code': '17421', 'title': '쇼생크 탈출'},
 {'code': '154667', 'title': '덕구'},
 {'code': '10200', 'title': '터미네이터 2'},
 {'code': '69105', 'title': '월-E'},
 {'code': '10016', 'title': '나 홀로 집에'},
 {'code': '18988', 'title': '라이언 일병 구하기'},
 {'code': '22126', 'title': '인생은 아름다워'},
 {'code': '24452', 'title': '매트릭스'},
 {'code': '10102', 'title': '사운드 오브 뮤직'},
 {'code': '136900', 'title': '어벤져스: 엔드게임'},
 {'code': '10002', 'title': '빽 투 더 퓨쳐'},
 {'code': '17159', 'title': '포레스트 검프'},
 {'code': '82432', 'title': '헬프'},
 {'code': '92125', 'title': '헌터 킬러'},
 {'code': '29217', 'title': '글래디에이터'},
 {'code': '35901', 'title': '살인의 추억'},
 {'code': '66463', 'title': '토이 스토리 3'},
 {'code': '32686', 'titl

In [7]:
def get_movie_info(movie):
    code = movie.get('code')
    title = movie.get('title')
    
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code': code
    }
    
    resp = requests.get(base_url, params)
    soup = BeautifulSoup(resp.content)
    
    open_date_tag = soup.find('dl', class_='info_spec').find('dd').find_all('span')[-1]
    open_date = ""
    
    for date in open_date_tag.find_all('a'):
        open_date += date.text
        
    open_date = "-".join(open_date.strip().split('.'))
    
    
    story = soup.find('div', class_= 'story_area').text
    
    movie['story'] = story
    movie['open_date'] = open_date
    
    return movie

In [8]:
movie_lst = get_movie_list()
movie_lst[0]

get_movie_info(movie_list[0])

NameError: name 'movie_list' is not defined

In [9]:
# query = """
#         CREATE TABLE IF NOT EXISTS naver_movie
#             (code    INTEGER      NOT NULL      PRIMARY KEY,
#              created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
#              title   VARCHAR(100) NOT NULL,
#              story   TEXT,
#              released_date  DATETIME)
#              ENGINE INNODB;
#         """
def insert_movie(movie):
    code = movie.get('code')
    title = movie.get('title')
    story = movie.get('story')
    open_date = movie.get('open_date')
    query = """
    INSERT INTO naver_movie
    (code, title, story, released_date)
    VALUES('{0}', '{1}', %s, '{2}')
    """.format(code, title, story, open_date)
    cur.execute(query, [story])
    db.commit()
    

In [10]:
sample_movie = get_movie_info(movie_lst[0])
sample_movie

{'code': '171539',
 'title': '그린 북',
 'story': '\n\n줄거리\n\n언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’\r\n원칙보다 반칙! 다혈질 운전사 ‘토니’\r\n취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!\n1962년 미국, 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 우아함 그 자체인\r\xa0천재 피아니스트 돈 셜리(마허샬라 알리) 박사의 운전기사 면접을 보게 된다.\r\xa0\r\xa0백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는\r\xa0위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고,\r\xa0투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다.\r\xa0\r\xa0거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사.\r\xa0생각, 행동, 말투, 취향까지 달라도 너무 다른 두 사람은\r\xa0그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…\n제작노트 보기\n',
 'open_date': '2019-01-09'}

In [11]:
insert_movie(sample_movie)

OperationalError: (1366, "Incorrect string value: '\\xEA\\xB7\\xB8\\xEB\\xA6\\xB0...' for column `naver_movie`.`naver_movie`.`title` at row 1")

In [12]:
def get_comment_list(code, last_page):
    url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn'
    params = {
        'code' : code,
        'type' : 'after',
        'order' : 'newest'
    }
    result_list = []
    for page in range(1, last_page+ 1):
        params['page'] = page
        
        resp = requests.get(url, params)
        resp.encoding='utf8'
        soup = BeautifulSoup(resp.content)
        
        reple_list = soup.find('div', class_ = 'score_result').find_all('li')
        
        for reple in reple_list:
            score = reple.find(class_ = 'star_score').text.strip()
            content = reple.find('p')
            name_n_date = content.find_next_sibling('dl').find_all('em')
            content = content.text
            
            nickname = name_n_date[0].text
            date = name_n_date[1]
            result_list.append({
                'write_name' : nickname,
                'write_date' : date.text,
                'content' : content,
                'score' : score,
                'code' : code
            })
    return result_list

In [13]:
get_comment_list('171539', 2)

[{'write_name': '\n\n봉봉(lemo****)\n\n',
  'write_date': '2019.06.17 00:07',
  'content': '따뜻한 브로맨스 로드무비.  ',
  'score': '10',
  'code': '171539'},
 {'write_name': '\n\nWindy(appl****)\n\n',
  'write_date': '2019.06.15 21:46',
  'content': '평점 클라스 봐라 ㅋㅋ 너무한거 아니냐?  ',
  'score': '1',
  'code': '171539'},
 {'write_name': '\n\nlunasea(luna****)\n\n',
  'write_date': '2019.06.15 17:51',
  'content': '블루레이로 감삼.두 배우의 연기가 대단하고 내용 전개 중에 심한 갈등이 나올거 같았는데 나오지 않아 편하고 재밌게 볼수 있었다.  ',
  'score': '10',
  'code': '171539'},
 {'write_name': '\n\n임소현(mira****)\n\n',
  'write_date': '2019.06.15 11:50',
  'content': '노잼 노잼 노잼 노잼 노잼  ',
  'score': '1',
  'code': '171539'},
 {'write_name': '\n\n신검무적 진산월(bajj****)\n\n',
  'write_date': '2019.06.14 23:43',
  'content': '시작은 유쾌했다, 그렇지만 가면 갈수록 묘하게 안타까우면서 슬프고 화가났지만, 토니의 현란한 말빨로 상황을 넘어가며 관객과 배우는 아슬아슬한 줄타기를 이어나갔다, 그러면서 서로를 배우게 되는 두남자의 우정과 빠꾸없는 직진이 맘에들었다. 감독덤앤더머감독  ',
  'score': '10',
  'code': '171539'},
 {'write_name': '\n\nokcu****\n\n',
  'write_date': '2019.06.

In [14]:
def insert_comment(comment):
    score = comment.get('score')
    writer = comment.get('writer')
    write_date = comment.get('write_date')
    
    write_date = '-'.join(write_date.split('.'))
    
    content = comment.get('content')
    movie_code = comment.get('code')
    
    query = """
    INSERT INTO movie_comment(score, writer, written_at, content, movie_code)
    VALUES('{}', '{}','{}', '{}','{}')
    """.format(score,writer,write_date, content, movie_code)
    print(query)
    cur.execute(query)
    db.commit()

In [15]:
comment_list = get_comment_list('171539', 3)
for comment in comment_list:
    insert_comment(comment)


    INSERT INTO movie_comment(score, writer, written_at, content, movie_code)
    VALUES('10', 'None','2019-06-17 00:07', '따뜻한 브로맨스 로드무비.  ','171539')
    


OperationalError: (1366, "Incorrect string value: '\\xEB\\x94\\xB0\\xEB\\x9C\\xBB...' for column `naver_movie`.`movie_comment`.`content` at row 1")

In [ ]:
for movie in movie_lst:
    try:
        comment_list = get_comment_list(movie['code'])
        for comment in comment_list:
            insert_comment(comment)
            
    except Exception:
        pass